In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from omegaconf import OmegaConf

from typing import Tuple

from boozie.get_data import load_wine

In [3]:
cfg = OmegaConf.load("../boozie/conf/config.yaml")

In [4]:
df = load_wine()

In [5]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
df.quality.value_counts().sort_index()

quality
3      30
4     216
5    2138
6    2836
7    1079
8     193
9       5
Name: count, dtype: int64

In [7]:
def extract_samples(
    df: pd.DataFrame, samples: dict, random_state: int = 42
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    df = df.copy()
    samples_df = pd.DataFrame()

    for name, score in samples.items():
        subset = df[df["quality"] == score]

        record = (subset.sample(random_state=random_state)
                        .assign(name=name))
        df = df.drop(record.index)
        samples_df = pd.concat([samples_df, record])

    return samples_df, df

In [8]:
samples_df, tdf = extract_samples(df, cfg.samples)

In [9]:
samples_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,name
2419,6.6,0.36,0.29,1.6,0.021,24.0,85.0,0.98965,3.41,0.61,12.4,9,Schmidt
1114,5.0,0.40,0.50,4.3,0.046,29.0,80.0,0.99020,3.49,0.66,13.6,6,Penfolds
107,6.2,0.63,0.31,1.7,0.088,15.0,64.0,0.99690,3.46,0.79,9.3,5,Antinori
5821,6.1,0.38,0.14,3.9,0.060,27.0,113.0,0.99344,3.07,0.34,9.2,4,Vega Sicilia
5008,6.2,0.23,0.35,0.7,0.051,24.0,111.0,0.99160,3.37,0.43,11.0,3,S-Budget


In [15]:
tdf.loc[2420, :]

fixed acidity             7.3000
volatile acidity          0.6550
citric acid               0.2000
residual sugar           10.2000
chlorides                 0.0710
free sulfur dioxide      28.0000
total sulfur dioxide    212.0000
density                   0.9971
pH                        2.9600
sulphates                 0.5800
alcohol                   9.2000
quality                   6.0000
Name: 2420, dtype: float64